In [42]:
# import defaults
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [43]:
# options for dealing with websites that populate while a user scrolls
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)


In [51]:
driver = webdriver.Chrome() #options=chrome_options

# Open the page
driver.get("https://www.google.com/search?q=GameStop&sca_esv=54b6785aa301342b&tbs=cdr:1,cd_min:1/4/2021,cd_max:12/31/2021&tbm=nws&source=lnms&sa=X&ved=2ahUKEwjjn4HW1qaEAxV5FFkFHYNAAZ4Q0pQJegQIBhAG&biw=1600&bih=731&dpr=1")

# Wait for the initial content to load
time.sleep(5)  # Adjust this delay as necessary

counter = 0
all_titles = []
all_dates = []

# Scroll down to load more articles
html = driver.find_element(By.TAG_NAME, 'html')
for _ in range(10):  # Adjust the range depending on how much you need to scroll
    html.send_keys(Keys.END)
    

#Source: https://sqa.stackexchange.com/questions/50747/selenium-python-write-click-next-page-loop
#set counter to 25; Google typically allows only 30 pages of results for this type of query
#counter is not synced with Selenium operations or timer
#Google also occassionally changes the XPATH to the next button
while counter <=25:
    # Scrape the article titles
    articles = driver.find_elements(By.XPATH, "//body//a[@class='WlydOe']//div[@class='n0jPhd ynAwRc MBeuO nDgy9d']")
    pub_dates = driver.find_elements(By.XPATH, "//div[@class='SoAPf']/div[@class='OSrXXb rbYSKb LfVVr']/span")
    titles = [article.text for article in articles]
    dates = [date.text for date in pub_dates]
    for title in titles:
        all_titles.append(title)
    for date in dates:
        all_dates.append(date)
    #Get the "Next" button using the XPath
    next_button_xpath = "/html/body[@id='gsr']/div[@id='main']/div[@id='cnt']/div[@id='rcnt']/div[@id='center_col']/div[@id='botstuff']/div/div[3]/table[@class='AaVjTc']/tbody/tr/td[@class='d6cvqb BBwThe'][2]/a[@id='pnnext']/span[2]"
    next_button = driver.find_element(By.XPATH, next_button_xpath)
    
    # Click Next
    next_button.click()
    counter +=1
    time.sleep(2)  # Add delay between scrolls for the page to load
    

# Close the browser
driver.quit()

In [52]:
len(all_titles)

260

In [53]:
len(all_dates)

260

In [54]:
dict = {'Date': all_dates, 'Title': all_titles}
df = pd.DataFrame(dict)
#Source: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Title
0,2021-01-28,GameStop Stock Soars as Reddit Investors Take ...
1,2021-02-13,GameStop timeline: A closer look at the saga t...
2,2021-01-30,Controversy Over GameStop’s Stock Market Saga ...
3,2021-02-03,Remembering the GameStop of My Childhood | SUP...
4,2021-01-30,Brockton native Keith Gill caused GameStop $GM...


In [60]:
#Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
df = df.sort_values(by='Date', ascending=True)
df.head()

,Date,Title
30,2021-01-25,The GameStop Game Never Stops
22,2021-01-26,The GameStop Fiasco Proves We're in a 'Meme St...
259,2021-01-27,The complete moron's guide to GameStop's stock...
153,2021-01-27,Steven Cohen's Point72 invests $750 million in...
6,2021-01-28,"The Whole Messy, Ridiculous GameStop Saga in O..."


In [56]:
df.isna().sum()

Date     0
Title    0
dtype: int64

In [61]:
df.to_csv("GME_news_data.csv", index=False)